In [113]:
import json
import requests
from geoalchemy2 import Geometry
import psycopg2
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import URL
import numpy as np 
import plotly.graph_objects as go


In [114]:
url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password="postgres",  # plain (unescaped) text
    host="localhost",
    database="postgres",
)

In [115]:
engine = create_engine(url_object)

In [116]:
site_table = pd.read_sql('SELECT * FROM meta.site ', engine)

In [117]:
site_table

,id,code,name,site_type_id,owner_id,addr_id,geom_latlon,description,parent_id,name_eng,lon,lat
0,3,6005,р.Амур - с.Джалинда,2,245.0,142.0,0101000020E6100000056EDDCD53F95E407FF6234564BC...,None,NaN,None,123.895750,53.471810
1,4,6010,р.Амур - с.Черняево,2,NaN,142.0,0101000020E61000001C5F7B6649805F404F5DF92CCF63...,None,NaN,None,126.004480,52.779760
2,5,6016,р.Амур - с.Кумара,2,NaN,142.0,0101000020E6100000D9CEF753E3AD5F407F6ABC7493C8...,,NaN,None,126.717000,51.567000
3,6,6020,р.Амур - с.Сергеевка,2,NaN,142.0,0101000020E61000006B9A779CA2D35F40C0046EDDCD5F...,None,NaN,None,127.306800,50.748470
4,7,6022,р.Амур - г.Благовещенск,2,NaN,142.0,0101000020E6100000B8E4B8533AE05F405CC98E8D4020...,None,NaN,None,127.503560,50.251970
...,...,...,...,...,...,...,...,...,...,...,...,...
10228,30848,Сахшел2,Порт Холмск,15,1329.0,177.0,0101000020E6100000C52BFF1011C16140666666666686...,None,NaN,Kholmsk port,142.033333,47.050000
10229,30849,Сахшел3,Залив Анива,15,1329.0,177.0,0101000020E61000003333333333D36140DA7D5C555555...,None,NaN,Aniva bay,142.600000,46.666667
10230,30850,Сахшел4,Месторождение на шельфе о. Сахалин,15,1329.0,177.0,0101000020E61000006666666666F661409A9999999939...,None,NaN,Sakhalin-Shelf oilfield,143.700000,52.450000
10231,30851,Сахшел5,Залив Терпения,15,1329.0,177.0,0101000020E6100000CDCCCCCCCC1C62409A99999999B9...,None,NaN,Sakhalin-Shelf oilfield,144.900000,47.450000


In [118]:
id_len = len(site_table['id'])
code_len = len(site_table['code'])
name_len = len(site_table['name'])
site_type_id_len = len(site_table['site_type_id'])
owner_id_len = len(site_table['owner_id'])
addr_id_len = len(site_table['addr_id'])
geom_latlon_len = len(site_table['geom_latlon'])
description_len = len(site_table['description'])
parent_id_len = len(site_table['parent_id'])
name_eng_len = len(site_table['name_eng'])
lon_len = len(site_table['lon'])
lat_len = len(site_table['lat'])

int_len = id_len + site_type_id_len + owner_id_len + addr_id_len
string_len = code_len + name_len + description_len + name_eng_len
geo_len = geom_latlon_len
float_len = lon_len + lat_len

In [119]:
unique_site = site_table.nunique()
unique_site = pd.DataFrame(unique_site, columns=[''])
# unique_site = unique_site.drop(columns=["geom_latlon", 'lon', 'lat'])

In [120]:
unique_site.T

,id,code,name,site_type_id,owner_id,addr_id,geom_latlon,description,parent_id,name_eng,lon,lat
,10233,9737,9932,43,41,77,9318,116,94,181,5937,4724


In [121]:
labels = list(unique_site.T)
values = unique_site.T.values.tolist()

fig = go.Figure(data=[go.Bar(x = labels, y = values[0])])
fig.show()

In [122]:
fig = go.Figure()

site_table['id'].value_counts()

fig.add_trace(go.Pie(values=[int_len, string_len, geo_len, float_len], labels=["int4", 'string', 'geometry', 'float'], hole=0.9))

fig.update_layout(
    annotations=[dict(text='Количество<br>переменных<br>по типу', x=0.5, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [123]:
unique_site.T.drop(columns = 'geom_latlon')

,id,code,name,site_type_id,owner_id,addr_id,description,parent_id,name_eng,lon,lat
,10233,9737,9932,43,41,77,116,94,181,5937,4724


In [124]:
site_table

,id,code,name,site_type_id,owner_id,addr_id,geom_latlon,description,parent_id,name_eng,lon,lat
0,3,6005,р.Амур - с.Джалинда,2,245.0,142.0,0101000020E6100000056EDDCD53F95E407FF6234564BC...,None,NaN,None,123.895750,53.471810
1,4,6010,р.Амур - с.Черняево,2,NaN,142.0,0101000020E61000001C5F7B6649805F404F5DF92CCF63...,None,NaN,None,126.004480,52.779760
2,5,6016,р.Амур - с.Кумара,2,NaN,142.0,0101000020E6100000D9CEF753E3AD5F407F6ABC7493C8...,,NaN,None,126.717000,51.567000
3,6,6020,р.Амур - с.Сергеевка,2,NaN,142.0,0101000020E61000006B9A779CA2D35F40C0046EDDCD5F...,None,NaN,None,127.306800,50.748470
4,7,6022,р.Амур - г.Благовещенск,2,NaN,142.0,0101000020E6100000B8E4B8533AE05F405CC98E8D4020...,None,NaN,None,127.503560,50.251970
...,...,...,...,...,...,...,...,...,...,...,...,...
10228,30848,Сахшел2,Порт Холмск,15,1329.0,177.0,0101000020E6100000C52BFF1011C16140666666666686...,None,NaN,Kholmsk port,142.033333,47.050000
10229,30849,Сахшел3,Залив Анива,15,1329.0,177.0,0101000020E61000003333333333D36140DA7D5C555555...,None,NaN,Aniva bay,142.600000,46.666667
10230,30850,Сахшел4,Месторождение на шельфе о. Сахалин,15,1329.0,177.0,0101000020E61000006666666666F661409A9999999939...,None,NaN,Sakhalin-Shelf oilfield,143.700000,52.450000
10231,30851,Сахшел5,Залив Терпения,15,1329.0,177.0,0101000020E6100000CDCCCCCCCC1C62409A99999999B9...,None,NaN,Sakhalin-Shelf oilfield,144.900000,47.450000


In [125]:
str(site_table['lon'].dtypes)


'float64'

In [126]:
site_table.describe()

,id,site_type_id,owner_id,addr_id,parent_id,lon,lat
count,10233.000000,10233.000000,4170.000000,3802.000000,352.000000,9560.000000,9560.000000
mean,22334.072022,24.149516,870.889928,176.098369,23095.178977,99.443170,51.893920
std,8399.741800,45.264660,536.718249,77.636338,8942.642183,43.867532,8.300936
min,2.000000,1.000000,0.000000,1.000000,3.000000,-179.508523,16.833300
25%,21332.000000,2.000000,246.000000,136.000000,24211.750000,53.472500,46.250000
50%,24447.000000,3.000000,1307.000000,166.000000,27728.000000,113.560694,52.050000
75%,27018.000000,12.000000,1318.000000,264.000000,27729.000000,137.000000,56.820045
max,30852.000000,200.000000,1329.000000,271.000000,27729.000000,190.250000,80.750000


In [146]:
site_table["site_type_id"]

0         2
1         2
2         2
3         2
4         2
         ..
10228    15
10229    15
10230    15
10231    15
10232    15
Name: site_type_id, Length: 10233, dtype: int64

In [150]:


fig = go.Figure(go.Scattermapbox(lat=site_table["lat"], lon=site_table["lon"], text=site_table["name"]))
map_center = go.layout.mapbox.Center(lat=58.38, 
                                     lon=97.45)
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=2))
fig.show()

In [133]:
site_table["name"]

0                       р.Амур - с.Джалинда
1                       р.Амур - с.Черняево
2                         р.Амур - с.Кумара
3                      р.Амур - с.Сергеевка
4                   р.Амур - г.Благовещенск
                        ...                
10228                           Порт Холмск
10229                           Залив Анива
10230    Месторождение на шельфе о. Сахалин
10231                        Залив Терпения
10232                    Пролив Асланбекова
Name: name, Length: 10233, dtype: object